# Model: Model for Overflow of Xu et al. 1999

###### This file has been automatically generated with the Template Generator
###### Here is the link to the GitHub repository: https://git.tu-berlin.de/bvt-htbd/modelling/e-coli-model-overview/
###### Version 1.7.2 from 30.09.25 Matteo Di Fiore, New Feature: Better log messages
###### Name of the Paper: Xu et al. - 1999 - Modeling of Overflow Metabolism in Batch and Fed-Batch Cultures of Escherichia coli
###### DOI: 10.1021/bp9801087
###### Description and assumptions: This is the model for overflow metabolism of E. coli as described in the paper of Xu et al. 1999. The model includes substrate uptake, acetate production and consumption, oxygen uptake, and biomass growth. The model is based on Monod kinetics with inhibition terms for acetate and oxygen. The model can simulate both batch and fed-batch cultures, with the option to include a feeding strategy based on substrate concentration. The model also includes a maintenance term for biomass growth. The model runs with if clauses to see if acetate is produced or consumed.
###### Goal of Simulation: Recreate the results of the paper and provide a basis for further model extensions.
###### Created on 2025 Jul Wed
###### Updated on 30.09.2025
###### @authors: Matteo Di Fiore

In [1]:
# Used packages
import numpy as np                                         # Numerical computing, arrays, linear algebra
from scipy.integrate import odeint                         # ODE solver for integrating differential equations
from math import exp                                       # Exponential function (for kinetics and growth rates)
import class_plot as cplt                                  # Custom plotting class (data visualization)
import class_fedbatch as cfb                               # Custom fed-batch model class (feeding & pulse logic)
import class_allthedata as catd                            # Custom all-the-data model class (dictionary creation)
from markdown_generation import generate_markdown_table    # Custom markdown generation function
from IPython.display import Markdown                       # Display formatted Markdown in Jupyter notebooks
import os                                                  # System coding (to save folders)

In [2]:
# Time variables
t0 = 0.0 # initial time of simulation in h
dt = 0.01 # steps of simulation in h
t_end = 15 # change if you want, this is the time of the end of simulation in h
t_constant_feed = 14 # change if you want, this is the time of the beginning of continuos feeding in h
t_fedbatch_start = 6 # change if you want, this is the time of the beginning of fedbatch in h
t = np.linspace(t0, t_end, int(t_end/dt)+1)

In [3]:
# Feed options

# this value decides when the feeding starts based on the remaining Substrate
substrate_limit_fedbatch = 0.00001    # this value is in (g/L) change the value if you want

# initializing the feeding class
# you can call the feeding class with t_fedbatch_start, substrate_limit_fedbatch, t_constant_feed and any combination of them
# just remember that if you use both t_fedbatch_start and substrate_limit_fedbatch the feeding will start when the t_fedbatch_start is reached
# if you want a fedbatch (with the feed start happening at t_fedbatch) it should be like this: feed = cfb.Feed(t_fedbatch_start = t_fedbatch_start), just remember to define t_fedbatch_start with a number
# if you want a fedbatch (with the feed start happening at substrate_limit_fedbatch) it should be like this: feed = cfb.Feed(substrate_limit = substrate_limit_fedbatch), just remember to attribute a number to substrate_limit_fedbatch (you can find it at the button of this code block)
# if you want a fedbatch (with the feed start happening at t_fedbatch or substrate_limit_fedbatch) and a constant feed it should be like this: feed = cfb.Feed(t_const = t_constant_feed, t_fedbatch_start = t_fedbatch_start) or feed = cfb.Feed(t_const = t_constant_feed, substrate_limit = substrate_limit_fedbatch) , just remember to define teh variables with a number
feed = cfb.Feed(t_const = t_constant_feed, substrate_limit = substrate_limit_fedbatch) # write here the variable which determine how you feed. In this case it is a batch simulation

In [4]:
#change to false if you want the units to be g/(g h)
q_o_unit_mmol = True

In [5]:
variables = dict(
    Biomass = dict(
        description = 'Biomass',
        unit = '[g/L]',
        initial_value = 0.1,
        boundaries = [None, None],
        estimable = None,
        weight = None,
        plotting = {'plot': None, 'range_fedbatch': None},
        volume_related = None,
        conversion_factor = None),
    Substrate = dict(
        description = 'Substrate',
        unit = '[g/L]',
        initial_value = 13.86,
        boundaries = [None, None],
        estimable = None,
        weight = None,
        plotting = {'plot': None, 'range_fedbatch': None},
        volume_related = None,
        conversion_factor = None),
    Acetate = dict(
        description = 'Acetate',
        unit = '[g/L]',
        initial_value = 0,
        boundaries = [None, None],
        estimable = None,
        weight = None,
        plotting = {'plot': None, 'range_fedbatch': None},
        volume_related = None,
        conversion_factor = None),
    Volume = dict(
        description = 'Volume',
        unit = '[L]',
        initial_value = 7,
        boundaries = [None, None],
        estimable = None,
        weight = None,
        plotting = {'plot': None, 'range_fedbatch': None},
        volume_related = None,
        conversion_factor = None),
)

In [6]:
# For the Variables
md_output = generate_markdown_table(variables, 'Variables')
# Display in Jupyter
Markdown(md_output)

## Variables

| name | Lower boundary | Upper boundary | conversion_factor | description | estimable | initial_value | unit | volume_related | weight |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Biomass | None | None | None | Biomass | None | 0.1 | [g/L] | None | None |
| Substrate | None | None | None | Substrate | None | 13.86 | [g/L] | None | None |
| Acetate | None | None | None | Acetate | None | 0 | [g/L] | None | None |
| Volume | None | None | None | Volume | None | 7 | [L] | None | None |

## Variables - plotting specs

| Name | plot | range_fedbatch |
| --- | --- | --- |
| Biomass | None | None |
| Substrate | None | None |
| Acetate | None | None |
| Volume | None | None |


In [7]:
inputs = dict(
    Feed = dict(
        description = 'Feed',
        unit = '[L]',
        concentration = 520,
        plotting = {'plot': None, 'stem': None, 'range': [None, None]}),
)

In [8]:
# For the Inputs
md_output = generate_markdown_table(inputs, 'Inputs')
# Display in Jupyter
Markdown(md_output)

## Inputs

| name | concentration | description | unit |
| --- | --- | --- | --- |
| Feed | 520 | Feed | [L] |

## Inputs - plotting specs

| Name | plot | range | stem |
| --- | --- | --- | --- |
| Feed | None | [None, None] | None |


In [9]:
parameters = dict(
)

In [10]:
constants = dict(
    q_s_max = dict(value = 1.3, unit = '[g/(g*h)]', description = 'Maximum substrate uptake rate'),
    q_o_max_mmol = dict(value = 15.6, unit = '[mmol/(g*h)]', description = 'Maximum oxygen uptake rate in mmol'),
    molar_mass_O2 = dict(value = 32, unit = '[g/mol]', description = 'Molar mass of oxygen'),
    q_o_max = dict(value = 0.4992, unit = '[g/(g*h)]', description = 'Maximum oxygen uptake rate'),
    q_a_c_max_b = dict(value = 0.06, unit = '[g/(g*h)]', description = 'Maximum acetate consumption rate (batch)'),
    q_a_c_max_f = dict(value = 0.15, unit = '[g/(g*h)]', description = 'Maximum acetate consumption rate (fed-batch)'),
    K_s = dict(value = 0.05, unit = '[g/L]', description = 'Substrate affinity constant'),
    K_a = dict(value = 0.05, unit = '[g/L]', description = 'Acetate inhibition constant'),
    K_io = dict(value = 4, unit = '[g/L]', description = 'Oxygen inhibition constant'),
    K_is = dict(value = 5, unit = '[g/L]', description = 'Substrate inhibition constant'),
    Y_xs_ox = dict(value = 0.51, unit = '[g/g]', description = 'Oxidative biomass yield on substrate'),
    Y_xs_of = dict(value = 0.15, unit = '[g/g]', description = 'Overflow biomass yield on substrate'),
    Y_xa = dict(value = 0.4, unit = '[g/g]', description = 'Acetate yield on biomass'),
    Y_os = dict(value = 1.067, unit = '[g/g]', description = 'Oxygen yield on substrate'),
    Y_oa = dict(value = 1.067, unit = '[g/g]', description = 'Oxygen yield on acetate'),
    Y_as = dict(value = 0.667, unit = '[g/g]', description = 'Acetate yield on substrate'),
    q_m = dict(value = 0.04, unit = '[g/(g*h)]', description = 'Maintenance rate'),
    C_s = dict(value = 1/30, unit = '[mol C/g]', description = 'Carbon content in substrate'),
    C_a = dict(value = 1/30, unit = '[mol C/g]', description = 'Carbon content in acetate'),
    C_x = dict(value = 0.04, unit = '[mol C/g]', description = 'Carbon content in biomass'),
    mu_set = dict(value = 0.3, unit = '[1/h]', description = 'Set specific growth rate'),
    G = dict(value = 1.1, unit = '[-]', description = 'Stoichiometric coefficient'),
)


In [11]:
# For the Constants
md_output = generate_markdown_table(constants, 'Constants')
# Display in Jupyter
Markdown(md_output)

## Constants

| name | description | unit | value |
| --- | --- | --- | --- |
| q_s_max | Maximum substrate uptake rate | [g/(g*h)] | 1.3 |
| q_o_max_mmol | Maximum oxygen uptake rate in mmol | [mmol/(g*h)] | 15.6 |
| molar_mass_O2 | Molar mass of oxygen | [g/mol] | 32 |
| q_o_max | Maximum oxygen uptake rate | [g/(g*h)] | 0.4992 |
| q_a_c_max_b | Maximum acetate consumption rate (batch) | [g/(g*h)] | 0.06 |
| q_a_c_max_f | Maximum acetate consumption rate (fed-batch) | [g/(g*h)] | 0.15 |
| K_s | Substrate affinity constant | [g/L] | 0.05 |
| K_a | Acetate inhibition constant | [g/L] | 0.05 |
| K_io | Oxygen inhibition constant | [g/L] | 4 |
| K_is | Substrate inhibition constant | [g/L] | 5 |
| Y_xs_ox | Oxidative biomass yield on substrate | [g/g] | 0.51 |
| Y_xs_of | Overflow biomass yield on substrate | [g/g] | 0.15 |
| Y_xa | Acetate yield on biomass | [g/g] | 0.4 |
| Y_os | Oxygen yield on substrate | [g/g] | 1.067 |
| Y_oa | Oxygen yield on acetate | [g/g] | 1.067 |
| Y_as | Acetate yield on substrate | [g/g] | 0.667 |
| q_m | Maintenance rate | [g/(g*h)] | 0.04 |
| C_s | Carbon content in substrate | [mol C/g] | 0.03333333333333333 |
| C_a | Carbon content in acetate | [mol C/g] | 0.03333333333333333 |
| C_x | Carbon content in biomass | [mol C/g] | 0.04 |
| mu_set | Set specific growth rate | [1/h] | 0.3 |
| G | Stoichiometric coefficient | [-] | 1.1 |


In [12]:
rates = dict(
    q_s=dict(unit='[g/g/h]', description='Specific glucose uptake rate (Monod + acetate inhibition)'),
    q_s_ox=dict(unit='[g/g/h]', description='Total glucose uptake rate into oxidative metabolism'),
    q_s_ox_an=dict(unit='[g/g/h]', description='Glucose uptake rate for anabolic part of oxidative metabolism'),
    q_s_ox_en=dict(unit='[g/g/h]', description='Glucose uptake rate for energetic part of oxidative metabolism'),
    q_o_s=dict(unit='[g/g/h]', description='Oxygen uptake rate from glucose oxidation'),
    q_s_of=dict(unit='[g/g/h]', description='Glucose uptake rate for overflow metabolism'),
    q_s_of_an=dict(unit='[g/g/h]', description='Glucose uptake rate for anabolic part of overflow metabolism'),
    q_s_of_en=dict(unit='[g/g/h]', description='Glucose uptake rate for energetic part of overflow metabolism'),
    q_a_p=dict(unit='[g/g/h]', description='Acetate production rate from overflow metabolism'),
    q_a_c=dict(unit='[g/g/h]', description='Acetate uptake rate (Michaelis-Menten kinetics)'),
    q_a_c_an=dict(unit='[g/g/h]', description='Acetate uptake rate used for biomass synthesis'),
    q_a_c_en=dict(unit='[g/g/h]', description='Acetate uptake rate for energy use constrained by oxygen availability'),
    q_a=dict(unit='[g/g/h]', description='Net acetate accumulation rate (production minus consumption)'),
    q_o=dict(unit='depends on q_o_unit_mmol', description='Total oxygen uptake rate (glucose oxidation + acetate energy use)'),
    my=dict(unit='[1/h]', description='Specific growth rate combining anabolic contributions from glucose and acetate'),
    OCR=dict(unit='[mmol/(g*h)]', description='Oxygen consumption rate in mmol')
)


In [13]:
# For the Rates
md_output = generate_markdown_table(rates, 'Rates')
# Display in Jupyter
Markdown(md_output)

## Rates

| name | description | unit |
| --- | --- | --- |
| q_s | Specific glucose uptake rate (Monod + acetate inhibition) | [g/g/h] |
| q_s_ox | Total glucose uptake rate into oxidative metabolism | [g/g/h] |
| q_s_ox_an | Glucose uptake rate for anabolic part of oxidative metabolism | [g/g/h] |
| q_s_ox_en | Glucose uptake rate for energetic part of oxidative metabolism | [g/g/h] |
| q_o_s | Oxygen uptake rate from glucose oxidation | [g/g/h] |
| q_s_of | Glucose uptake rate for overflow metabolism | [g/g/h] |
| q_s_of_an | Glucose uptake rate for anabolic part of overflow metabolism | [g/g/h] |
| q_s_of_en | Glucose uptake rate for energetic part of overflow metabolism | [g/g/h] |
| q_a_p | Acetate production rate from overflow metabolism | [g/g/h] |
| q_a_c | Acetate uptake rate (Michaelis-Menten kinetics) | [g/g/h] |
| q_a_c_an | Acetate uptake rate used for biomass synthesis | [g/g/h] |
| q_a_c_en | Acetate uptake rate for energy use constrained by oxygen availability | [g/g/h] |
| q_a | Net acetate accumulation rate (production minus consumption) | [g/g/h] |
| q_o | Total oxygen uptake rate (glucose oxidation + acetate energy use) | depends on q_o_unit_mmol |
| my | Specific growth rate combining anabolic contributions from glucose and acetate | [1/h] |
| OCR | Oxygen consumption rate in mmol | [mmol/(g*h)] |


In [14]:
# Equations
def algebra(x, t):


   
    


    '''------------------- generated variables -------------------'''

    X = x[0]      # Biomass
    S = x[1]    # Substrate
    A = x[2]   # Acetate
    V = x[3]    # Volume

   

    '''------------------- generated constants -------------------'''

    q_s_max = constants['q_s_max']['value']    # Maximum substrate uptake rate
    q_o_max_mmol = constants['q_o_max_mmol']['value']   # Maximum oxygen uptake rate in mmol
    molar_mass_O2 = constants['molar_mass_O2']['value']    # Molar mass of oxygen
    q_o_max = constants['q_o_max']['value']     # Maximum oxygen uptake rate
    q_a_c_max_b = constants['q_a_c_max_b']['value']     # Maximum acetate consumption rate (batch)
    q_a_c_max_f = constants['q_a_c_max_f']['value']    # Maximum acetate consumption rate (fed-batch)
    K_s = constants['K_s']['value']   # Substrate affinity constant
    K_a = constants['K_a']['value']    # Acetate inhibition constant
    K_io = constants['K_io']['value']    # Oxygen inhibition constant
    K_is = constants['K_is']['value']    # Substrate inhibition constant
    Y_xs_ox = constants['Y_xs_ox']['value']    # Oxidative biomass yield on substrate
    Y_xs_of = constants['Y_xs_of']['value']    # Overflow biomass yield on substrate
    Y_xa = constants['Y_xa']['value']    # Acetate yield on biomass
    Y_os = constants['Y_os']['value']    # Oxygen yield on substrate
    Y_oa = constants['Y_oa']['value']    # Oxygen yield on acetate
    Y_as = constants['Y_as']['value']    # Acetate yield on substrate
    q_m = constants['q_m']['value']    # Maintenance rate
    C_s = constants['C_s']['value']    # Carbon content in substrate
    C_a = constants['C_a']['value']    # Carbon content in acetate
    C_x = constants['C_x']['value']    # Carbon content in biomass    
    mu_set = constants['mu_set']['value']    # Set specific growth rate
    G = constants['G']['value']    # Stoichiometric coefficient

    '''------------------- feed options  -------------------'''


    #use S_feed as your feed concentration in your Formulas
    S_feed = inputs['Feed']['concentration']

    # this is an optional feature you can  use if you want some changes depending on the feeding state
    if feed.batch:
        # put here the things that should happen while we dont feed
        pass
    if feed.feeding_first:
        # put here the things that should happen when feeding starts (will happen only once)
        pass # delete this if not needed but remember!: you can call with feed.ts the time when the feed started and with feed.X_fedbatch_start the Biomass at the moment of feed start
    if feed.feeding_exp:
        q_a_c_max_b = q_a_c_max_f
        #set G to 1.3
        G = 1.3
    if feed.feeding_const:
        pass # put here your induction equation(s)

    '''------------------- put here your equations  -------------------'''

    # algebraic equations
    # [g/g/h]  -- specific glucose uptake rate (Monod + acetate inhibition) (eq 1)
    q_s = q_s_max / (1 + A / K_is) * S / (S + K_s)  
    
    # [g/g/h]  -- total glucose taken into oxidative metabolism
    q_s_ox = q_s  
    
    # [g/g/h]  -- specific glucose uptake rate for anabolic part of oxidative metabolism (eq 2)
    # q_m [g/g/h], Y_xs_ox [g/g], C_x [mol C/g], C_s [mol C/g]
    q_s_ox_an = (q_s_ox - q_m) * Y_xs_ox * C_x / C_s
    
    # [g/g/h]  -- specific glucose uptake rate for energetic part of oxidative metabolism (eq 3)
    q_s_ox_en = q_s_ox - q_s_ox_an  
    
    # [g/g/h]  -- specific glucose uptake rate for oxygen demand from glucose oxidation (eq 4)
    # Y_os [g/g]
    q_o_s = q_s_ox_en * Y_os  
    
    # Dynamic oxygen limit [g/g/h]
    q_o_lim = q_o_max / (1 + A / K_io)
    
    if q_o_s >= q_o_lim:
        # proportional reduction of oxidative flux to satisfy condition
        q_s_ox_en_scaling = q_s_ox_en
        q_s_ox_en = q_o_lim / Y_os 
        scaling_s = q_s_ox_en / q_s_ox_en_scaling           
        q_s_ox_an *= scaling_s
    
        q_o_s = q_o_lim
    
        # [g/g/h] -- specific glucose uptake rate for the oxidative flux (eq 5)
        q_s_ox = q_s_ox_an + q_s_ox_en  
    
        # [g/g/h]  -- specific glucose uptake rate for overflow metabolism (eq 6)
        q_s_of = q_s - q_s_ox  
    
        # [g/g/h]  -- specific glucose uptake rate for anabolic part of overflow metabolism (eq 7)
        q_s_of_an = q_s_of * Y_xs_of * C_x / C_s   
    
        # [g/g/h]  -- specific glucose uptake rate for energetic part of overflow metabolism (eq 8)   
        q_s_of_en = q_s_of - q_s_of_an  
    
        # [g/g/h]  -- specific acetate production from overflow metabolism (eq 9)
        # Y_as [g/g]    
        q_a_p = q_s_of_en * Y_as
    
        # no acetate consumption during overflow
        q_a_c = 0
        q_a_c_en = 0
        q_a_c_an = 0
        q_a_c_en_constrain  = 0
    
    else:
        # no overflow metabolism, no acetate production
        q_a_p = 0
        q_s_of = 0
        q_s_of_en = 0
        q_s_of_an = 0
    
        # [g/g/h]  -- specific acetate uptake (Michaelis-Menten, eq 10)
        q_a_c = q_a_c_max_b * (A / (A + K_a))  
    
        # [g/g/h]  -- specific acetate used for biomass synthesis (eq 11)
        # C_x [mol C/g], C_a [mol C/g]
        q_a_c_an = q_a_c * Y_xa * C_x / C_a 
    
        # [g/g/h] -- specific acetate for energy-use, constrained by oxygen demand (eq 12)
        q_a_c_en_constrain  =  (q_o_max - q_o_s) / Y_oa
    
        q_a_c_en = min((q_a_c - q_a_c_an), q_a_c_en_constrain)
    
        if (q_a_c - q_a_c_an) >= q_a_c_en_constrain:
            scaling_a = q_a_c_en_constrain / (q_a_c - q_a_c_an)
            q_a_c *= scaling_a
            q_a_c_an *= scaling_a
    
    # [g/g/h] -- specific acetate accumulation (production - consumption)    
    q_a = q_a_p - q_a_c
    
    # [g/g/h] -- specific oxygen consumption (glucose oxidation + acetate energy use), (eq 13)   
    q_o = (q_o_s + q_a_c_en * Y_oa) / G
    
    # [1/h] -- specific growth rate (mu), combines anabolic contributions of q_s_ox, q_s_of, and q_a_c (eq 14)
    my = (q_s_ox - q_m) * Y_xs_ox + q_s_of * Y_xs_of + q_a_c * Y_xa

    OCR = q_o / 32 * X * 1000


    #remeber to add the variables you want to plot and need for your differential equations that are not rates
    return {
    'q_s': q_s, 'q_s_ox': q_s_ox, 'q_s_ox_an': q_s_ox_an, 'q_s_ox_en': q_s_ox_en, 'q_o_s': q_o_s,
    'q_s_of': q_s_of, 'q_s_of_an': q_s_of_an, 'q_s_of_en': q_s_of_en, 'q_a_p': q_a_p, 'q_a_c': q_a_c,
    'q_a_c_an': q_a_c_an, 'q_a_c_en': q_a_c_en, 'q_a': q_a, 'q_o': q_o, 'my': my, 'OCR': OCR
    }


In [15]:
# Function for the hole Model
def dif_simple(x, t):




    '''------------------- generated variables -------------------'''

    X = x[0]    # Biomass
    S = x[1]    # Substrate
    A = x[2]    # Acetate
    V = x[3]    # Volume

    

    '''------------------- generated constants -------------------'''

    q_s_max = constants['q_s_max']['value']    # Maximum substrate uptake rate
    q_o_max_mmol = constants['q_o_max_mmol']['value']    # Maximum oxygen uptake rate in mmol
    molar_mass_O2 = constants['molar_mass_O2']['value']    # Molar mass of oxygen
    q_o_max = constants['q_o_max']['value']    # Maximum oxygen uptake rate
    q_a_c_max_b = constants['q_a_c_max_b']['value']    # Maximum acetate production rate (batch)
    q_a_c_max_f = constants['q_a_c_max_f']['value']    # Maximum acetate production rate (fed-batch)
    K_s = constants['K_s']['value']    # Substrate affinity constant
    K_a = constants['K_a']['value']    # Acetate inhibition constant
    K_io = constants['K_io']['value']    # Oxygen inhibition constant
    K_is = constants['K_is']['value']    # Substrate inhibition constant
    Y_xs_ox = constants['Y_xs_ox']['value']    # Oxidative biomass yield on substrate
    Y_xs_of = constants['Y_xs_of']['value']    # Overflow biomass yield on substrate
    Y_xa = constants['Y_xa']['value']    # Acetate yield on biomass
    Y_os = constants['Y_os']['value']    # Oxygen yield on substrate
    Y_oa = constants['Y_oa']['value']    # Oxygen yield on acetate
    Y_as = constants['Y_as']['value']    # Acetate yield on substrate
    q_m = constants['q_m']['value']    # Maintenance rate
    C_s = constants['C_s']['value']    # Carbon content in substrate
    C_a = constants['C_a']['value']    # Carbon content in acetate
    C_x = constants['C_x']['value']    # Carbon content in biomass
    mu_set = constants['mu_set']['value']    # Set specific growth rate
    G = constants['G']['value']    # Stoichiometric coefficient

    rs = algebra(x, t)

    '''------------------- generated rates -------------------'''

    q_s = rs['q_s']    #Specific substrate uptake rate
    q_o_s = rs['q_o_s']    #Specific oxygen uptake rate on substrate
    q_a_p = rs['q_a_p']    #Specific acetate production rate (product)
    q_a_c = rs['q_a_c']    #Specific acetate consumption rate
    q_a = rs['q_a']    #Total specific acetate rate
    q_a_c_en = rs['q_a_c_en']    #Specific acetate rate under energy limitation
    q_o = rs['q_o']    #Total specific oxygen uptake rate
    my = rs['my']    #Specific growth rate
    OCR = rs['OCR']    #Oxygen consumption rate in mmol/(L*h)


    '''------------------- feed options  -------------------'''

    feed.get(t, S, X)    # S and X are the variables for Substrate and Biomass change them if needed

    #use S_feed as your feed concentration in your Formulas
    S_feed = inputs['Feed']['concentration']
    V0 = variables['Volume']['initial_value'] # import V0 as initial volume

    # this is an optional feature you can  use if you want some changes depending on the feeding state
    if feed.batch:
        # put here the things that should happen while we dont feed
        F = 0
    if feed.feeding_first:
        # put here the things that should happen when feeding starts (will happen only once)

        F0 = V0 * feed.X_fedbatch_start * mu_set / (Y_xs_ox * S_feed)
        F = F0 * exp(mu_set * (t - feed.ts))
        print(F0, F0 * exp(mu_set * (t - feed.ts)), t)
    if feed.feeding_exp:
        F0 = V0 * feed.X_fedbatch_start * mu_set / (Y_xs_ox * S_feed)
        #calculate F
        F = F0 * exp(mu_set * (t - feed.ts))
            
    if feed.feeding_const:
        F = 0.188
    '''------------------- put here your differential equations -------------------'''
    dXdt = (-F / V + my) * X         
    dSdt = F / V * (S_feed - S) - q_s  * X        
    dVdt = F 
    dAdt = (q_a_p - q_a_c)  * X - (F / V) * A


    #this will be used by odeint
    dxdt = [dXdt, dSdt, dAdt, dVdt]
    return dxdt

In [16]:
# Define initial conditions
x0 = [variables['Biomass']['initial_value'], variables['Substrate']['initial_value'], variables['Acetate']['initial_value'], variables['Volume']['initial_value']]

x = odeint(dif_simple, x0, t)
# Print the feed start time once
print("Feed started at:", feed.ts)

Feeding started via substrate limit
Feeding started at time: 7.497
Feeding started at Biomass: 6.321461
0.05005681462304764 0.05005681462304764 7.496935860147105
Feed started at: 7.496935860147105


In [17]:
# Actual integration phase by phase for more smoothness

if feed.t_const is None and feed.t_fedbatch_start is None and feed.substrate_limit is None:
    mode = 'Mode: Batch only'
elif feed.t_const is not None and feed.t_fedbatch_start is None and feed.substrate_limit is None:
    t_1 = np.linspace(t0, t_constant_feed, int(t_constant_feed/dt)+1)
    t_2 = np.linspace(t_constant_feed, t_end, int((t_end - t_constant_feed)/dt)+1)
    feed = cfb.Feed(t_const = t_constant_feed)
    x_1 = odeint(dif_simple, x0, t_1)
    x_2 = odeint(dif_simple, x_1[-1], t_2)
    x = np.vstack((x_1[:-1], x_2))
    mode = 'Mode: Constant feed only'
elif feed.t_const is None and feed.t_fedbatch_start is not None and feed.substrate_limit is None:
    t_1 = np.linspace(t0, t_fedbatch_start, int(t_fedbatch_start/dt)+1)
    t_2 = np.linspace(t_fedbatch_start, t_end, int((t_end - t_fedbatch_start)/dt)+1)
    feed = cfb.Feed(t_fedbatch_start = t_fedbatch_start)
    x_1 = odeint(dif_simple, x0, t_1)
    x_2 = odeint(dif_simple, x_1[-1], t_2)
    x = np.vstack((x_1[:-1], x_2))
    mode = 'Mode: Fedbatch only (start by time)'
elif feed.t_const is None and feed.t_fedbatch_start is None and feed.substrate_limit is not None:
    t_1 = np.linspace(t0, feed.ts, int(feed.ts/dt)+1)
    t_2 = np.linspace(feed.ts, t_end, int((t_end - feed.ts)/dt)+1)
    feed = cfb.Feed(substrate_limit = substrate_limit_fedbatch)
    x_1 = odeint(dif_simple, x0, t_1)
    x_2 = odeint(dif_simple, x_1[-1], t_2)
    x = np.vstack((x_1[:-1], x_2))
    mode = 'Mode: Fedbatch only (start by substrate limit)'
elif feed.t_const is not None and feed.t_fedbatch_start is not None and feed.substrate_limit is None:
    t_1 = np.linspace(t0, t_fedbatch_start, int(t_fedbatch_start/dt)+1)
    t_2 = np.linspace(t_fedbatch_start, t_constant_feed, int((t_constant_feed - t_fedbatch_start)/dt)+1)
    t_3 = np.linspace(t_constant_feed, t_end, int((t_end - t_constant_feed)/dt)+1)
    feed = cfb.Feed(t_const = t_constant_feed, t_fedbatch_start = t_fedbatch_start)
    x_1 = odeint(dif_simple, x0, t_1)
    x_2 = odeint(dif_simple, x_1[-1], t_2)
    x_3 = odeint(dif_simple, x_2[-1], t_3)
    x = np.vstack((x_1[:-1], x_2[:-1], x_3))
    mode = 'Mode: Const. feed + fedbatch (by time)'
elif feed.t_const is not None and feed.t_fedbatch_start is None and feed.substrate_limit is not None:
    t_1 = np.linspace(t0, feed.ts, int(feed.ts/dt)+1)
    t_2 = np.linspace(feed.ts, t_constant_feed, int((t_constant_feed - feed.ts)/dt)+1)
    t_3 = np.linspace(t_constant_feed, t_end, int((t_end - t_constant_feed)/dt)+1)
    feed = cfb.Feed(t_const = t_constant_feed, substrate_limit = substrate_limit_fedbatch)
    x_1 = odeint(dif_simple, x0, t_1)
    x_2 = odeint(dif_simple, x_1[-1], t_2)
    x_3 = odeint(dif_simple, x_2[-1], t_3)
    x = np.vstack((x_1[:-1], x_2[:-1], x_3))
    mode = 'Mode: Const. feed + fedbatch (by substrate)'
elif feed.t_const is None and feed.t_fedbatch_start is not None and feed.substrate_limit is not None:
    t_1 = np.linspace(t0, feed.ts, int(feed.ts/dt)+1)
    t_2 = np.linspace(feed.ts, t_end, int((t_end - feed.ts)/dt)+1)
    feed = cfb.Feed(t_fedbatch_start = t_fedbatch_start, substrate_limit = substrate_limit_fedbatch)
    x_1 = odeint(dif_simple, x0, t_1)
    x_2 = odeint(dif_simple, x_1[-1], t_2)
    x = np.vstack((x_1[:-1], x_2))
    mode = 'Mode: Fedbatch (start by time or substrate)'
elif feed.t_const is not None and feed.t_fedbatch_start is not None and feed.substrate_limit is not None:
    t_1 = np.linspace(t0, feed.ts, int(feed.ts/dt)+1)
    t_2 = np.linspace(feed.ts, t_constant_feed, int((t_constant_feed - feed.ts)/dt)+1)
    t_3 = np.linspace(t_constant_feed, t_end, int((t_end - t_constant_feed)/dt)+1)
    feed = cfb.Feed(t_const = t_constant_feed, t_fedbatch_start = t_fedbatch_start, substrate_limit = substrate_limit_fedbatch)
    x_1 = odeint(dif_simple, x0, t_1)
    x_2 = odeint(dif_simple, x_1[-1], t_2)
    x_3 = odeint(dif_simple, x_2[-1], t_3)
    x = np.vstack((x_1[:-1], x_2[:-1], x_3))
    mode = 'Mode: Const. feed + fedbatch (start by time or substrate)'
print(mode)

Feeding started via substrate limit
Feeding started at time: 7.497
Feeding started at Biomass: 6.321462
0.05005682504104443 0.05005682504104443 7.497021288255519
Mode: Const. feed + fedbatch (by substrate)


In [18]:
'''Here all the metabolic fluxes that will be plotet wil be added to x '''
algebra_results = []    #just to initiate the variable
q_m = constants['q_m']['value']    # Maintenance rate

# Iterate over time steps to access individual values
for state_vector, time_point in zip(x, t):
    #if you calculate the oxygen consumption rate calculate it here, because its not a differential equation
    X, S, A, V = state_vector
    results = algebra(state_vector, time_point)
    
    #set different qo if depending on the unit you choose
    if q_o_unit_mmol:
        q_o_mmol = results['q_o'] / 32 * 1000
        q_o_unit = "[mmol/(g·h)]"
        rates['q_o']['unit'] = q_o_unit
    else:
        q_o_mmol = results['q_o']
        q_o_unit = "[g/(g·h)]"
        rates['q_o']['unit'] = q_o_unit
    algebra_results.append([
        results['OCR'],
        results['q_s'],
        results['q_s_ox'],
        results['q_s_ox_an'],
        results['q_s_ox_en'],
        results['q_o_s'],
        results['q_s_of'],
        results['q_s_of_an'],
        results['q_s_of_en'],
        results['q_a_p'],
        results['q_a_c'],
        results['q_a_c_an'],
        results['q_a_c_en'],
        results['q_a'],
        q_o_mmol,
        results['my'],
    

    ])

#combine everything in x
algebra_array = np.array(algebra_results)
x_and_rs = np.hstack((x, algebra_array))

In [19]:
# Initializing the allthedata dictionary

if feed.t_const is None and feed.t_fedbatch_start is None and feed.substrate_limit is None:
    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t)
elif feed.t_const is not None and feed.t_fedbatch_start is None and feed.substrate_limit is None:
    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t, t_constant_feed = t_2)
elif feed.t_const is None and feed.t_fedbatch_start is not None and feed.substrate_limit is None:
    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t, t_2)
elif feed.t_const is None and feed.t_fedbatch_start is None and feed.substrate_limit is not None:
    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t, t_2)
elif feed.t_const is not None and feed.t_fedbatch_start is not None and feed.substrate_limit is None:
    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t, t_2, t_3)
elif feed.t_const is not None and feed.t_fedbatch_start is None and feed.substrate_limit is not None:
    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t, t_2, t_3)
elif feed.t_const is None and feed.t_fedbatch_start is not None and feed.substrate_limit is not None:
    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t, t_2)
elif feed.t_const is not None and feed.t_fedbatch_start is not None and feed.substrate_limit is not None:
    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t, t_2, t_3)

allthedata = builder.build()

In [20]:
path_to_results, datestring, run_id = builder.init_saving(1, 1, os.path.join(os.getcwdb().decode('utf-8'), 'Xu et al. Results'), None)
allthedata_plot = {"1": allthedata}
plot = cplt.PlotPlotly(
    allthedata_plot,
    path_to_results,
    plot_info=f"{datestring}_{run_id}",
    plot_mode="white_w_grid",
    colorscheme="matlab_default",
    plot_formats=("html")
)
plot.use_column_title = True
plot.plot_height_multiple_rows = 1000  # default is 200, increase for bigger plots
plot.plot_height_one_row = 600  # default is 225
plot.plot_by_expID(
    plot_replicates=False,
    show_legend=False,
    plot_SE_data=False,
    split_at_feed_start=False,
    range_end=None,
    plot_multiple_list=[
        dict(members=["Biomass", "Substrate"], axes=["left", "right"])
    ]
)

Results are saved here: C:\Users\matdf\Documents\Bachelorarbeit\git_tub\e-coli-model-overview\Library_of_models\ecoli_1999_Xu_Overflow\Xu et al. Results\20251009_16-31-05_1
